### Web Scrapping

In [1]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [62]:
lang = "bangla" # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "saad"
folder = f"{surah}-38"
file = f"{surah.lower()}_{lang.lower()}"

In [63]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

In [64]:
content = HTMLFile.read()

In [65]:
soup = BeautifulSoup(content, 'lxml')

In [66]:
print(soup.prettify())

<html>
 <body>
  <div class="ayahs-block">
   <div class="ayah-1 ayah-single 38" id="1">
    <div class="page-block" id="page-453">
     PAGE 453
    </div>
    <div class="row ayah-row">
     <div class="col-1 ayah-buttons">
      <span class="ayah-block-buttons ayah-number-button">
       Ayah 1
      </span>
      <span class="ayah-block-buttons play-ayah-button">
       Play Ayah
      </span>
      <span class="ayah-block-buttons stop-recitation-button" style="display: none;">
       Stop Recitation
      </span>
     </div>
     <div class="col-10 single-ayah 1">
      <span class="single-word" data-ts="" id="38-1-1">
       <span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">
        صٓۚ
       </span>
       <span class="word-transliteration">
        sad
       </span>
       <span class="word-translation">
        সাদ
       </span>
      </span>
      <span class="single-word" data-ts="" id="38-1-2">
       <span class="word-arabic word-arabic-u

In [67]:
ayah_word = soup.findAll('span', attrs={"class":"single-word"})
ayah_word

[<span class="single-word" data-ts="" id="38-1-1"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">صٓۚ</span> <span class="word-transliteration">sad</span><span class="word-translation">সাদ</span></span>,
 <span class="single-word" data-ts="" id="38-1-2"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">وَٱلۡقُرۡءَانِ</span> <span class="word-transliteration">wal-qur'ani</span><span class="word-translation">কোরআনের শপথ</span></span>,
 <span class="single-word" data-ts="" id="38-1-3"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">ذِي</span> <span class="word-transliteration">dhi</span><span class="word-translation">পূর্ণ</span></span>,
 <span class="single-word" data-ts="" id="38-1-4"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">ٱلذِّكۡرِ</span> <span class="word-transliteration">al-dhik'ri</span><span class="word-translation">উপদেশ</span></span>,
 <sp

In [68]:
word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    var1_list = re.findall(r'''(?<=>)(.*)(?=</span>)''', var1)
    #    arabic=" ".join(var1_list)
    #except:
    #    pass
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

In [69]:
word_list

[('صٓۚ', 'sad', 'সাদ'),
 ('وَٱلۡقُرۡءَانِ', "wal-qur'ani", 'কোরআনের শপথ'),
 ('ذِي', 'dhi', 'পূর্ণ'),
 ('ٱلذِّكۡرِ', "al-dhik'ri", 'উপদেশ'),
 ('بَلِ', 'bali', 'কিন্তু'),
 ('ٱلَّذِينَ', 'alladhina', 'যারা'),
 ('كَفَرُواْ', 'kafaru', 'অস্বীকার করেছে'),
 ('فِي', 'fi', 'মধ্যে (লিপ্ত)'),
 ('عِزَّةٖ', 'izzatin', 'ঔদ্ধত্যের'),
 ('وَشِقَاقٖ', 'washiqaqin', 'ও বিরোধীতার'),
 ('كَمۡ', 'kam', 'কত (জাতিকেই)'),
 ('أَهۡلَكۡنَا', 'ahlakna', 'আমরা ধ্বংস করেছি'),
 ('مِن', 'min', 'পূর্বে'),
 ('قَبۡلِهِم', 'qablihim', 'তাদের'),
 ('مِّن', 'min', 'মধ্যে হতে'),
 ('قَرۡنٖ', 'qarnin', 'জাতিসমূহের'),
 ('فَنَادَواْ', 'fanadaw', 'তারা তখন চিৎকার করেছে'),
 ('وَّلَاتَ', 'walata', 'কিন্তু আর ছিলো না'),
 ('حِينَ', 'hina', 'সময়'),
 ('مَنَاصٖ', 'manasin', 'মুক্তি লাভের'),
 ('وَعَجِبُوٓاْ', "wa'ajibu", 'এবং অবাক হয়েছে তারা'),
 ('أَن', 'an', 'যে'),
 ('جَآءَهُم', 'jaahum', 'তাদের কাছে এসেছে'),
 ('مُّنذِرٞ', 'mundhirun', 'একজন সতর্ককারী'),
 ('مِّنۡهُمۡۖ', "min'hum", "তাদেরই মধ্য হ'তে"),
 ('وَقَالَ', 'waqala', 'এবং বললো')

In [70]:
df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
# df_word_list = df_word_list.iloc[3:,] for all body element
df_word_list

,arabic,transliterate,translate_bangla
0,صٓۚ,sad,সাদ
1,وَٱلۡقُرۡءَانِ,wal-qur'ani,কোরআনের শপথ
2,ذِي,dhi,পূর্ণ
3,ٱلذِّكۡرِ,al-dhik'ri,উপদেশ
4,بَلِ,bali,কিন্তু
...,...,...,...
728,لِّلۡعَٰلَمِينَ,lil'alamina,বিশ্ববাসীদের জন্যে
729,وَلَتَعۡلَمُنَّ,walata'lamunna,এবং তোমরা অবশ্যই জানবে
730,نَبَأَهُۥ,naba-ahu,তার খবর
731,بَعۡدَ,ba'da,পরেই


In [71]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge All Translate

In [72]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Saad_Arabic_Transliterate_Translate_Words_Indonesian.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Saad_Arabic_Transliterate_Translate_Words_Bangla.xlsx']

In [73]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,صٓۚ,sad,Saad,ص,ص,shad,সাদ,Sâd,Сад.
1,وَٱلۡقُرۡءَانِ,wal-qur'ani,By the Quran,قسم ہے قرآن کی،,क़सम है क़ुरआन,demi Al Quran,কোরআনের শপথ,Kur'an'a andolsun,"Клянусь Кораном,"
2,ذِي,dhi,full (of) reminder,والے,*,mempunyai,পূর্ণ,sahibi,обладающим
3,ٱلذِّكۡرِ,al-dhik'ri,full (of) reminder,نصحیت,नसीहत वाले की,peringatan,উপদেশ,şan şeref',напоминанием
4,بَلِ,bali,Nay,بلکہ,बल्कि,bahkan,কিন্তু,doğrusu,Однако
...,...,...,...,...,...,...,...,...,...
728,لِّلۡعَٰلَمِينَ,lil'alamina,to the worlds,جہان والوں کے لئے,तमाम जहान वालों के लिए,bagi semesta alam,বিশ্ববাসীদের জন্যে,bütün alemlere,"для миров,"
729,وَلَتَعۡلَمُنَّ,walata'lamunna,And surely you will know,اور البتہ تم ضرور جان لو گے,और अलबत्ता तुम ज़रूर जान लोगे,dan sungguh kamu akan mengetahui,এবং তোমরা অবশ্যই জানবে,gayet iyi bileceksiniz,И непременно вы узнаете
730,نَبَأَهُۥ,naba-ahu,its information,خبر اس کی,ख़बर उसकी,beritanya,তার খবর,onun haberini,весть (истинности) его
731,بَعۡدَ,ba'da,after,بعد,बाद,sesudah,পরেই,sonra,после


In [74]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)